In [ ]:
%run src\load_and_process_text.py

In [1]:
%run src\create_features_tfidf.py

In [1]:
#Loading csv with processed text and csv with feature_vecs (100 word embedding vectors + dummy variables for essay set)
import pandas as pd
import numpy as np
df = pd.read_csv('data\interim\processed_text.csv', index_col=0)
X = pd.read_csv('data\interim\\feature_vecs.csv')

In [2]:
df.head()

,essay_set,essay,domain1_score,rating_discrepancy,cleaned_text
essay_id,,,,,
1,1,"Dear local newspaper, I think effects computer...",8.0,0.333333,dear local newspaper think effect computer peo...
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9.0,0.416667,dear believe use computer benefit us many way ...
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7.0,0.333333,dear people use computer everyone agree benefi...
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10.0,0.416667,dear local newspaper find many expert say comp...
5,1,"Dear @LOCATION1, I know having computers has a...",8.0,0.333333,dear know computer positive effect people comp...


In [3]:
choice(list(set(df.index)))

NameError: name 'choice' is not defined

In [21]:
from random import choice
num = choice(list(set(df.index)))
df.loc[num, 'essay']

'I think That The author concludes the story with This paragraph to show that she does not want to give up on fighting her homesickness. The first time she bought that plant, it brought back memories of her old home and it showed how much she missed it there. By saying shell do the test again shows that she doesnt want to let ths defeat her in her new home and her new life.'

In [22]:
df.loc[num, 'cleaned_text']

'think author conclude story paragraph show want give fight homesickness first time buy plant bring back memory old home show much miss say shell test show want let th defeat new home new life'

### Scaling features and target variables
Starting with scaling features first, since I think it should be easier.

In [3]:
from sklearn.preprocessing import MinMaxScaler
scaler_x = MinMaxScaler()
X_scaled = scaler_x.fit_transform(X)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


Scaling target variable now.  Separate fit and transform scaler for each essay set since they are on different scales.

In [4]:
scaler_y = MinMaxScaler()

essay_scores = df.loc[:,'domain1_score'].values.reshape(-1,1)
y_scaled = scaler_y.fit_transform(essay_scores)
y_scaled = np.concatenate(y_scaled).flatten()

In [ ]:
scaler_y = MinMaxScaler()
y_scaled = []

for set_id in range(1,9):
    essay_scores = df.loc[df['essay_set']==set_id,'domain1_score'].values.reshape(-1,1)
    scaled_scores = scaler_y.fit_transform(essay_scores)
    y_scaled.append(scaled_scores)

y_scaled = np.concatenate(y_scaled).flatten()

In [5]:
y_scaled.shape

(12973,)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled)

In [7]:
X_train.shape

(9729, 282)

In [17]:
pd.DataFrame(X_train).describe()

,0,1,2,3,4,5,6,7,8,9,...,272,273,274,275,276,277,278,279,280,281
count,9729.000000,9729.000000,9729.000000,9729.000000,9729.000000,9729.000000,9729.000000,9729.000000,9729.000000,9729.000000,...,9729.000000,9729.000000,9729.000000,9729.000000,9729.000000,9729.000000,9729.000000,9729.000000,9729.000000,9729.000000
mean,0.498890,0.135471,0.141124,0.135163,0.136396,0.137835,0.138966,0.119128,0.055915,0.019333,...,0.014868,0.009318,0.025055,0.020541,0.070692,0.014734,0.013334,0.017285,0.013871,0.013966
std,0.287764,0.342244,0.348168,0.341915,0.343227,0.344745,0.345929,0.323956,0.229770,0.064669,...,0.054691,0.042882,0.075287,0.063329,0.104788,0.058280,0.061079,0.064760,0.064987,0.061325
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.250694,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.496377,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.746454,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.112858,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.999769,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.788180,1.000000,1.000000,0.665114,1.000000,1.000000,1.000000,0.878040,0.959007,1.000000


In [8]:
y_train.shape

(9729,)

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
param_grid = {'alpha':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}
ridge = Ridge(fit_intercept=False)
ridge_cv = GridSearchCV(ridge, param_grid, cv=3)
ridge_cv.fit(X_train, y_train)

In [ ]:
ridge_cv.best_params_

In [ ]:
ridge_cv.best_score_

In [ ]:
from sklearn.ensemble import RandomForestRegressor

randfor = RandomForestRegressor()
randfor.fit(X_train,y_train)

In [ ]:
randfor.feature_importances_

In [ ]:
randfor.score(X_test, y_test)

In [ ]:
y_pred = randfor.predict(X_test)

In [ ]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score()

In [ ]:
residuals = y_test - y_pred

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.distplot(residuals)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(y_test, y_pred)
plt.show()

In [ ]:
sns.distplot(y_test)

In [ ]:
sns.distplot(y_pred)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
param_grid = {'min_samples_split':[10, 50, 100, 500, 1000],
              'max_depth': [2, 4, 6, 8, 10, 12, 14]}

randfor = RandomForestRegressor(verbose=True)
randfor_cv = GridSearchCV(randfor, param_grid, cv=5)
randfor_cv.fit(X_train, y_train)

In [ ]:
randfor_cv.best_params_

In [ ]:
randfor_cv.best_score_

In [ ]:
from sklearn.svm import SVR

svr = SVR()
svr.fit(X_train,y_train)
svr.score(X_test, y_test)

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
param_grid = {'alpha':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}
lasso = Lasso()
lasso_cv = GridSearchCV(lasso, param_grid, cv=5)
lasso_cv.fit(X_train, y_train)

In [ ]:
lasso_cv.best_params_

In [ ]:
lasso_cv.best_score_

In [ ]:
lasso = Lasso(alpha=0.0001)
lasso.fit(X_train,y_train)
lasso.coef_

Examples from https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 50, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
rf_random.best_params_

In [10]:
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV

params = {'n_estimators':[50, 100, 500, 1000], 'max_depth':[2], 'min_samples_split': [2],
          'learning_rate':[1, 0.1, 0.3, 0.01], 'loss': ['ls']}

gbr = ensemble.GradientBoostingRegressor()
grid = GridSearchCV(gbr, params)
grid.fit(X_train, y_train)

y_pred = grid.predict(X_test)

# summarize the results of the grid search
print(grid.best_score_)
print(grid.best_estimator_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.9581661902678434
GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=500, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)


In [11]:
grid.best_score_

0.9581661902678434

In [12]:
grid.best_estimator_

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=500, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

AttributeError: 'GridSearchCV' object has no attribute 'get_important_features'

In [ ]:
mse = mean_squared_error(y_test, y_pred)
print("MSE: %.4f" % mse)

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % grid.score(X_test, y_test))
print()

In [14]:
y_pred.mean()

0.11504914806323215

In [20]:
df['domain1_score'].mean()

6.80104833114931

In [21]:
6.80/60

0.11333333333333333

ValueError: operands could not be broadcast together with shapes (12973,) (3244,) 

In [24]:
model = ensemble.GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=500, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [25]:
model.fit(X_scaled, y_scaled)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=500, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [26]:
model.score(X_scaled, y_scaled)

0.9744943517290129

In [27]:
y_scaled.mean()

0.11335080551915516

In [28]:
y_pred = model.predict(X_scaled)

In [29]:
y_pred.mean()

0.11335080551915515

In [33]:
diff = y_pred*60 - df['domain1_score']
import seaborn as sns
import matplotlib.pyplot as plt

sns.distplot(diff)
plt.show()

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


<Figure size 640x480 with 1 Axes>